In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% ! important;}
div.cell.code_cell.rendered{width:100%}
div.input_prompt{padding:0px}
div.CodeMirror {font-family:Consolas ; font-size:12pt;}
div.text_cell_render.rendered_html {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold}
div.input {font-family:Consolas ; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">Ollama_LLM으로 EXAONE 4.0 1.2B 사용(LnagChain)</font></b>
# 테스트

In [3]:
# %conda install -c conda-forge openpyxl

In [23]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = 'sam860/exaone-4.0:1.2b')

In [24]:
import pandas as pd
data = pd.read_excel('data/test_labeling.xlsx')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      557 non-null    datetime64[ns]
 1   제목      557 non-null    object        
 2   본문      555 non-null    object        
 3   링크      557 non-null    object        
 4   산업      557 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 21.9+ KB


In [6]:
data.loc[0,'본문']

'미국 11월 소비자물가지수(CPI)가 전년 대비 2.7% 상승했다고 18일(현지시간) 미국 노동통계국이 밝혔다. 이는 전문가 전망치(3.1%)와 전월치(2.9%)를 모두 밑도는 수치이자 2021년 초 이후 가장 낮은 수준이다. 변동성이 큰 식품과 에너지를 제외한 근원 CPI는 전년 대비 2.6% 상승했다.'

In [7]:
result = llm.invoke(data.loc[0,'본문']+'의 내용을 요약해 줘')

In [8]:
test_data = result.content

In [9]:
test_talk = llm.invoke(test_data+'의 내용은 긍정적인지 부정적인지만 파악해 줘')

In [10]:
test_talk.content

'주어진 정보에 따르면, 11월 미국 소비자물가지수(CPI)가 전년 대비 **2.7% 상승**한 것은 전체 인플레이션률에서는 높은 수치였으나, 전문가 전망치(3.1%)보다 낮았고 전월치(2.9%)에도 미치지 못했습니다. 이로 인해 변동성이 큰 식품·에너지 제외 근원 CPI는 더 낮은 증가율(2.6%)을 기록하며 완화된 인플레이션 압력을 보였습니다.  \n\n### 종합적 평가:  \n1. **부정적 요소**:  \n   - 전반적인 CPI 상승이 전문가 예상보다 낮아 경제적 우려가 일부 존재합니다. 특히 에너지·식품 등 핵심 항목의 큰 하락으로 전체 물가 안정에는 긍정적 측면도 있었지만, 중장기적 기준으로는 추가적인 통화정책 필요성을 시사할 수 있습니다.  \n   - 2021년 이후 최저 수준이라는 점에서 소비 및 생산 비용 상승 억제 추세가 지속되고 있음을 반영합니다.  \n\n2. **긍정적 요소**:  \n   - 다른 근원(비식·비에너지)의 점진적인 조정으로 물가 변동성 감소 효과가 나타났습니다. 이는 중앙은행 정책 부담 완화로 이어질 수 있습니다.  \n\n결론적으로, 단기적으로는 완화된 인플레이션이지만, 구조적 문제나 글로벌 공급망 재편 등 배경 요인을 고려할 때 지속적인 모니터링이 필요합니다.'

In [23]:
test_file=pd.read_csv('data/테스트파일.csv')

test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   본문      72 non-null     object
 1   산업      72 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [12]:
results = []
for text in test_file[:10]['본문']:
    result = llm.invoke(text + " 내용을 요약하고 긍정적인지 부정적인지만 파악해 줘")
    results.append(result.content)

In [20]:
test_results=[]
for idx in range(len(results)):
    result = llm.invoke(results[idx] + '내용중에 긍정적인지 부정적인지 두 방향으로 나누지 말고 하나의 시선으로만 파악해줘 예를 들어 이 기사가 부정적인것 같다면 부정적이고 그 이유를 요약하는 식으로')
    test_results.append(result.content)

In [21]:
for idx in range(len(test_results)):
    print(test_results[idx])
    print("-"*50)

**감정 분석 요약: 부정적 측면 강조**  

해당 사고는 대규모 인명 피해와 구조적 결함으로 인해 명확한 책임 소재 논란을 낳았습니다. 기업의 공식 사과에도 불구하고, 원인 규명 과정에서 표면과 150m 거리에서 진행된 연결 통로 문제 등 근본적 위험 관리 소홀이 확인되면서 사회적 신뢰 회복에 어려움을 겪고 있습니다. 특히 작업 중 sudden structural failure(철근 붕괴)가 발생한 점과 인명 피해 규모를 고려할 때, 단순한 과실 이상의 시스템적 취약성 노출로 평가됩니다. 정부와 기업의 재발 방지 대책 마련 여부도 향후 유사 사고 예방력에 대한 의문을 남긴다.
--------------------------------------------------
이 사고는 인명 피해와 더불어 공사 지연, 교통 혼란 등 중대한 사회적·경제적 파장을 예상케 하는 사건이다. 특히 대형 크레인 사고로 인한 매몰 인부의 희생은 산업 현장에 대한 불신을 deepen시키고, 긴급 복구 비용이 추가될 우려가 커 부정적 결과가 계속될 가능성이 매우 높다.
--------------------------------------------------
### 사고 평가 (부정적 관점 중심)  

본 사고는 건설 현장에서 발생한 중대한 인명사고로, 철근 구조물 붕�라는 구조적 결함이 원인으로 지목되면서 근본적인 예방 시스템의 취약성과 관리 소홀 문제가 드러납니다. 신속한 대응은 필요했으나, 감리단의 초기 확인 미흡과 보수적인 수리 지시 등이 피해자 회복 과정에서 추가적 고통을 야기할 수 있었습니다. 특히 외국인 작업자 지원 미비와 경제적 손실 우려는 사회적 책임 측면에서도 우려되며, 단순한 응급 조치를 넘어 안전 문화 개선 방안 마련 없이는 재발 가능성이 높습니다. 현장 조사 완료 후에도 책임 소재 규명과 공정 개선이 지연될 경우 향후 유사 사고 리스크가 지속될 수 있어 근본적 보완이 시급합니다.
---------------------------------------------

In [24]:
results = []
for text in test_file['본문']:
    result = llm.invoke(text + '내용중에 긍정적인지 부정적인지 두 방향으로 나누지 말고 하나의 시선으로만 파악해줘 예를 들어 이 기사가 부정적인것 같다면 부정적이고 그 이유를 요약하는 식으로')
    results.append(result.content)

In [16]:
for idx in range(len(results)):
    print(results[idx])
    print("-"*50)

In [28]:
result_sentiment=[]
for idx in range(len(results)):
    result = llm.invoke(results[idx]+'내용을 읽고 긍정/중립/부정 중 하나의 단어를 선택해 줘. 예를 들어 긍정적인 내용이면 "긍정"만 출력하면 돼')
    result_sentiment.append(result.content)

In [48]:
sentiment_replace=[]
for idx in range(len(result_sentiment)):
    sentiment_replace.append(result_sentiment[idx].replace('"',"").replace(" ",""))
print(sentiment_replace)

['부정', '부정', '부정', '긍정', '긍정', '긍정', '부정', '긍정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '긍정', '부정', '긍정', '부정', '긍정', '긍정', '긍정', '긍정', '강화', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '긍정', '증명', '긍정', '긍정', '긍정', '긍정', '긍정', 'positive', '긍정', '긍정', '긍정', '긍정', 'positive', '긍정', '긍정', '긍정', 'positive', '긍정', '긍정']


In [51]:
set(sentiment_replace)

{'positive', '강화', '긍정', '부정', '증명'}

In [4]:
import csv

In [6]:
with open("data/감정분석전처리전.csv", "w", newline="\n", encoding="utf-8") as f: 
    writer = csv.writer(f) 
    writer.writerow(sentiment_replace)

In [11]:
with open('data/감정분석전처리전.csv','r', encoding='utf-8') as f:
    reader = csv.reader(f) 
    test_data = next(reader) 
print(test_data)

['부정', '부정', '부정', '긍정', '긍정', '긍정', '부정', '긍정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '긍정', '부정', '긍정', '부정', '긍정', '긍정', '긍정', '긍정', '강화', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '긍정', '증명', '긍정', '긍정', '긍정', '긍정', '긍정', 'positive', '긍정', '긍정', '긍정', '긍정', 'positive', '긍정', '긍정', '긍정', 'positive', '긍정', '긍정']


In [14]:
sentiment_result=[]
for idx in range(len(test_data)):
    if test_data[idx] not in('강화','증명','positive','negative'):
        sentiment_result.append(test_data[idx])
    elif test_data[idx] in('강화', 'positive'):
        sentiment_result.append('긍정')
    elif test_data[idx] == 'negative':
        sentiment_result.append('부정')
    else:
        sentiment_result.append('중립')

In [15]:
print(sentiment_result)

['부정', '부정', '부정', '긍정', '긍정', '긍정', '부정', '긍정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '긍정', '부정', '긍정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '부정', '긍정', '중립', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정', '긍정']


In [58]:
with open("data/감정분석전처리테스트.csv", "w", newline="\n", encoding="utf-8") as f: 
    writer = csv.writer(f) 
    writer.writerow(sentiment_result)

In [19]:
import pickle

# 피클 파일 불러오기
with open("data/648_20251201-20251221.pkl", "rb") as f:   # 'rb' = read binary
    loaded_data = pickle.load(f)

print(loaded_data)
# {'name': 'Alice', 'age': 25, 'hobbies': ['reading', 'cycling']}

                     날짜                                          제목  \
0   2025-12-01 06:36:00             [기자수첩]'주식빚투' 괜찮다?…부동산 '부메랑' 된다면   
1   2025-12-01 06:50:00         석화 구조조정 윤곽 잡히긴 하는데…여수·울산 '지지부진' 이유는   
2   2025-12-01 07:00:00                좁아진 '휠라' 입지…'안타 효과'로 돌파구 찾을까   
3   2025-12-01 07:10:00  [거버넌스워치] 가온그룹, 창업주 작고 뒤 28살 CEO 경영권 안정 안간힘   
4   2025-12-01 07:30:00    [거버넌스워치] ‘캐리어 에어컨’ 오텍그룹, “경영 참여” 주요주주 출현   
..                  ...                                         ...   
663 2025-12-21 13:00:00                      '말차' 유행인데…물 대신 마셔도 되나요   
664 2025-12-21 14:00:00                   'AI 날개' 달고 스마트 철도 시대 앞당긴다   
665 2025-12-21 15:00:00              SDV 전면에 세운 현대차…하드웨어 한계 넘으려는 까닭   
666 2025-12-21 16:00:00                [생활경제25]'K뷰티' 날았다…새 대장은 에이피알   
667 2025-12-21 16:25:00              고려아연·영풍, 美제련소 프로젝트 놓고 연일 공방 지속   

                                                    본문  \
0    올해 코스피는 4000포인트라는 전인미답의 고지를 밟았다. '코스피 5000' 달성...   
1    대산을 시작으로 석유화학업계의 대규모 구조조정이

# 실행코드

In [25]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = 'sam860/exaone-4.0:1.2b')
import pandas as pd
data = pd.read_pickle('data/industry_labeled.pickle')

In [27]:
data.reset_index(inplace=True)

In [30]:
data.drop(columns=['index'], inplace=True)

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9544 entries, 0 to 9543
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      9544 non-null   datetime64[ns]
 1   제목      9544 non-null   object        
 2   본문      9544 non-null   object        
 3   링크      9544 non-null   object        
 4   산업      9544 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 372.9+ KB


In [54]:
# 감정분석을 위해 본문열만 저장
sentimenting_data = data.iloc[:,2]

In [59]:
# 본문열을 리스트로 저장
sentimenting_list = [data for data in sentimenting_data]

In [83]:
# 시간이 오래걸리는 듯 하여 tag_list로 본문 분할 후 감정분석 예정
tag_list = [[] for _ in range(int(len(sentimenting_list)/100))]

for idx in range(int(len(sentimenting_list)/100)):
    for tag in range(100):
        tag_list[idx].append(sentimenting_list[idx*100 + tag])

In [84]:
len(tag_list[0])

100

In [87]:
# 우선 24일에 실행해 볼 코드
# for idx in range(len(tag_list[0])):
result_test=[]
for idx in range(10):
    result = llm.invoke(tag_list[0][idx]+'내용을 읽고 긍정/중립/부정 중 하나의 단어를 선택해 줘. 예를 들어 긍정적인 내용이면 "긍정"만 출력하면 돼')
    result_test.append(result.content)

In [85]:
# 저장된 본문들을 하나씩 감정분석(23일 한번에 하려다 시간을 허비함, 시간을 줄이기 위해 리스트를 나눠서 할 필요성이 있어보임)
result_sentiment=[]
for idx in range(len(sentimenting_list)):
    result = llm.invoke(sentimenting_list[idx]+'내용을 읽고 긍정/중립/부정 중 하나의 단어를 선택해 줘. 예를 들어 긍정적인 내용이면 "긍정"만 출력하면 돼')
    result_sentiment.append(result.content)

In [ ]:
result_sentimentiment

In [ ]:
# 테스트에선 긍정/중립/부정 외에 다른 단어들도 출력되어 긍정/중립/부정만 나오도록 단어를 수정하기위한 코드(아직 위의 코드의 실행결과를 모르기에 실행결과 확인 후 변경할 것)
sentiment_result=[]
for idx in range(len(test_data)):
    if test_data[idx] not in('강화','증명','positive','negative'):
        sentiment_result.append(test_data[idx])
    elif test_data[idx] in('강화', 'positive'):
        sentiment_result.append('긍정')
    elif test_data[idx] == 'negative':
        sentiment_result.append('부정')
    else:
        sentiment_result.append('중립')